In [2]:
# importing all necessary libraries to run the code
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
import re
# using the variable sw to hold all stopwords that are in English
sw = stopwords.words('english')

In [3]:
data = pd.read_csv("all.csv") 
data.head(1)

,comment,rating,past,stopwords_removal,reviewer,id,stemmed,fee,title,label,...,stopwords_removal_nltk,present_simple,dataSource,appId,date,sentiScore_pos,present_con,length_words,stopwords_removal_lemmatization,Exclude
0,"Besides the occasional crash, this is an amazi...",5,0,"besides occasional crash, this amazing product...",NaN,264,"besid the occa crash, thi is an amaz produc wi...",NaN,Almost perfect,Bug,...,"besides occasional crash, amazing product tons...",2,RE2014_app_and_play_store_apps,NaN,NaN,3,1,22,"besides occasional crash, this amaze product w...",NaN


In [4]:
from nltk.corpus import stopwords
#applying pre-processing steps to remove stopwords and words of size less than 2
data['comment'] = data['comment'].apply(lambda x: x.split())
wordsEng = stopwords.words('english')
data['comment'] = data['comment'].apply(lambda x:[item for item in x if item not in wordsEng])
data['comment'] = data['comment'].apply(lambda x: [w for w in x if len(w)>2])
data['comment'] = data['comment'].apply(lambda x: " ".join(x))
data.head(2)

,comment,rating,past,stopwords_removal,reviewer,id,stemmed,fee,title,label,...,stopwords_removal_nltk,present_simple,dataSource,appId,date,sentiScore_pos,present_con,length_words,stopwords_removal_lemmatization,Exclude
0,"Besides occasional crash, amazing product tons...",5,0,"besides occasional crash, this amazing product...",NaN,264,"besid the occa crash, thi is an amaz produc wi...",NaN,Almost perfect,Bug,...,"besides occasional crash, amazing product tons...",2,RE2014_app_and_play_store_apps,NaN,NaN,3,1,22,"besides occasional crash, this amaze product w...",NaN
1,"This could great app predictable, full bugs un...",1,1,"this could be great app if was predictable, bu...",NaN,111,"thi could be a gre ap if it was predictable, b...",NaN,Take a photo of your boarding pass,Bug,...,"could great app predictable, full bugs unpredi...",9,AppStore_Random,382698565,NaN,3,0,58,"this could be great app if be predictable, but...",NaN


In [5]:
data.reset_index(inplace=True)

In [6]:
# using the variable sw to hold all stopwords that are in English
sw = stopwords.words('english')
TEXT=[]
#Rating=[]
for i in range(len(data)):
    #if data['cetagory'][i]=='Racing':
    review = re.sub('[^a-zA-Z]', ' ',data['comment'][i])
    #review = re.sub('[/(){}\[\]\|@!,;]', ' ',data['reviews'][i])
    #review = re.sub('[^0-9a-zA-Z #+_♥️]', ' ',data['reviews'][i])#Remove bad symbols
    review = re.sub(r'\d+', '',review)
    review = review.lower()
    review = review.split()

    review = [token for token in review if token not in sw]
    review=' '.join(review)
    TEXT.append(review)
    #Rating.append(data['rating'][i])

In [18]:
# importing all necessary libraries to run the code
import re,string
import numpy as np
import pandas as pd
import keras_metrics
import tensorflow.keras
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,Embedding,GlobalMaxPooling1D
# using the variable sw to hold all stopwords that are in English
sw = stopwords.words('english')

In [19]:
#converting twitterSentiment[] list into dataframe for serving it to keras tokenizer
dataSetFinal = pd.DataFrame(np.array(TEXT))
dataSetFinal.head(2)

,0
0,besides occasional crash amazing product tons ...
1,could great app predictable full bugs unpredic...


In [20]:
tokenizer = tensorflow.keras.preprocessing.text.Tokenizer(num_words=2500, lower=True,split=' ',filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(dataSetFinal[0].values)
#print(tokenizer.word_index)  # To see the dicstionary
X = tokenizer.texts_to_sequences(dataSetFinal[0].values)
X = tensorflow.keras.preprocessing.sequence.pad_sequences(X)

In [11]:
from keras import optimizers
from keras import regularizers
from keras import backend as K
weight_decay = 1e-4
#Deep Learning Network Structure
model = Sequential()
model.add(Embedding(2500,100, input_length=X.shape[1]))
model.add(Conv1D(64, 5, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(64, 5, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(5, activation='softmax'))
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy','mae','mse',keras_metrics.precision(), keras_metrics.recall()])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 23, 100)           250000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 23, 64)            32064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 11, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 11, 64)            20544     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
__________

In [25]:
#LSTM
batch_size=64
Y = pd.get_dummies(data['rating']).values
X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y, test_size = 0.30)
#Here we train the Network.
pred=model.fit(X_train, Y_train, batch_size =batch_size, epochs =50, verbose =2,validation_data=(X_valid,Y_valid))
pred

Train on 2583 samples, validate on 1108 samples
Epoch 1/50
 - 135s - loss: 1.2644 - acc: 0.5947 - mean_absolute_error: 0.2538 - mean_squared_error: 0.1253 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.1311 - val_acc: 0.5975 - val_mean_absolute_error: 0.2231 - val_mean_squared_error: 0.1126 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
 - 109s - loss: 1.0551 - acc: 0.6148 - mean_absolute_error: 0.2122 - mean_squared_error: 0.1047 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.9953 - val_acc: 0.5975 - val_mean_absolute_error: 0.2080 - val_mean_squared_error: 0.1001 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
 - 67s - loss: 0.9010 - acc: 0.6353 - mean_absolute_error: 0.1838 - mean_squared_error: 0.0910 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.9695 - val_acc: 0.6444 - val_mean_absolute_error: 0.1966 - val_mean_squared_error: 0.0980 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/50
 - 84s - loss: 

KeyboardInterrupt: 

In [51]:
#LSTM
score=[]
score=model.evaluate(X_valid,Y_valid,verbose=2,batch_size=batch_size)
#keras.metrics.binary_accuracy(Y_valid,pred)
print("score: %.2f" %(score[0]))
print("validation accuracy: %.2f" % (score[1]))
print("Precision: %.2f" %(score[4]))
print("Recacll: %.2f" %(score[5]))
print("Mean Absolute Error: %.2f" % (score[2]))
print("Mean Squared Error: %.2f" % (score[3]))

score: 0.39
validation accuracy: 0.92
Precision: 0.93
Recacll: 0.96
Mean Absolute Error: 0.08
Mean Squared Error: 0.07
